第一天处理缺失值
1、看一下数据
2、看一下我们有多少缺失的数据点
3、找出数据丢失的原因
4、缺失值下降
5、补全缺失值

## 1、看一下数据

In [1]:
#  加载模块
import pandas as pd
import numpy as np

#  读取数据
nfl_data = pd.read_csv('/data/kaggle/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv')
sf_permits = pd.read_csv('/data/kaggle/Building_Permits.csv')

#  设置随机种子
np.random.seed(0)

C:\Users\hnuxz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\hnuxz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#  看nfl_data几行数据
nfl_data.sample(5)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
142828,2012-09-24,2012092400,3,1,1.0,09:25,10,3265.0,39.0,SEA,...,1.379042,0.522390,0.477610,0.562151,0.437849,0.522390,0.039762,-0.002897,0.042659,2012
69534,2010-11-14,2010111408,15,3,2.0,05:37,6,1237.0,46.0,NYG,...,NaN,0.053894,0.946106,0.085723,0.914277,0.053894,0.031830,NaN,NaN,2010
372672,2017-10-01,2017100111,1,1,2.0,14:23,15,3563.0,37.0,NYG,...,NaN,0.524307,0.475693,0.578703,0.421297,0.475693,-0.054396,NaN,NaN,2017
290484,2015-11-01,2015110107,18,3,NaN,15:00,15,1800.0,0.0,SF,...,NaN,0.893864,0.106136,0.889745,0.110255,0.893864,-0.004119,NaN,NaN,2015
206636,2013-11-17,2013111707,15,2,4.0,00:07,1,1807.0,5.0,PIT,...,NaN,0.260880,0.739120,0.279439,0.720561,0.739120,-0.018559,NaN,NaN,2013


In [4]:
#  看sf_permits10行数据
sf_permits.sample(10)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
78651,201503120624,8,otc alterations permit,03/12/2015,4273,029,2986,NaN,26th,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,9.0,Mission,94110.0,"(37.74972976527956, -122.40963202424835)",1374057173006
177779,201707071220,8,otc alterations permit,07/07/2017,0453,004A,950,NaN,Bay,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,2.0,Russian Hill,94109.0,"(37.80472551510833, -122.42280760820965)",1469546420378
64978,M529127,8,otc alterations permit,10/21/2014,0478,011,1290,NaN,Chestnut,St,...,NaN,NaN,NaN,NaN,NaN,2.0,Russian Hill,94109.0,"(37.80244997614665, -122.42443255018165)",1359656465959
165958,201704073507,8,otc alterations permit,04/07/2017,6507,003A,1134,NaN,Noe,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Noe Valley,94114.0,"(37.750866393629146, -122.43209633372915)",1458922436280
147902,201701237639,8,otc alterations permit,01/23/2017,0623,001,1755,NaN,Van Ness,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,2.0,Pacific Heights,94109.0,"(37.791925784456105, -122.42306858292103)",1450933235988
116034,201602290793,8,otc alterations permit,02/29/2016,4943,013,1046,NaN,Hollister,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,10.0,Bayview Hunters Point,94124.0,"(37.719378906009005, -122.39129146290892)",1414305176506
142629,M739147,8,otc alterations permit,11/15/2016,0215C,031,1333,NaN,Jones,St,...,NaN,NaN,NaN,NaN,NaN,3.0,Nob Hill,94109.0,"(37.79362102799777, -122.41488237355445)",1444405434576
56544,201407312694,8,otc alterations permit,07/31/2014,1924,091,1746,NaN,20th,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,4.0,Sunset/Parkside,94122.0,"(37.75528549833576, -122.47732318761126)",1350812393675
90962,201507020598,8,otc alterations permit,07/02/2015,1506,014,567,NaN,38th,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,1.0,Outer Richmond,94121.0,"(37.77813319742798, -122.49889439953233)",1387243102000
71567,201501024841,8,otc alterations permit,01/02/2015,1271,036,33,NaN,Carl,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,5.0,Haight Ashbury,94117.0,"(37.765695245477474, -122.44852485073525)",136658994926


## 2、看一下我们有多少缺失的数据点

In [20]:
#  查看nfl_data每列中缺失数据点的数量
missing_values_count = nfl_data.isnull().sum()
#  查看前十个
print(missing_values_count[0: 10])
#  查看有多少缺失值
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()
print('total_missing:' + str(total_missing) + ', total_cells:' + str(total_cells))
#  缺失数据百分比
(total_missing/total_cells) * 100

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64
total_missing:10342875, total_cells:41584176


24.87214126835169

In [21]:
#  查看sf_permits缺失点的适量
sf_missing_values_count = sf_permits.isnull().sum()
#  查看前十个
print(sf_missing_values_count[0: 10])
#  查看sf_permits有多少缺失值及百分比
sf_total_cells = np.product(sf_permits.shape)
sf_total_missing = sf_missing_values_count.sum()
(sf_total_missing / sf_total_cells) * 100

Permit Number                  0
Permit Type                    0
Permit Type Definition         0
Permit Creation Date           0
Block                          0
Lot                            0
Street Number                  0
Street Number Suffix      196684
Street Name                    0
Street Suffix               2768
dtype: int64


26.26002315058403

## 3找出数据丢失的原因

In [22]:
#  查看前十个缺失值
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

根据描述可以得知这是个描述游戏的表，缺失的时间值应该是没有记录，而不是不存在。所以这种可以猜测原因，而不是认为是空值

In [24]:
#  查看Street Number Suffix和Zipcode缺失值数量
sf_missing_values_count[['Street Number Suffix', 'Zipcode']]

Street Number Suffix    196684
Zipcode                   1716
dtype: int64

Street Number Suffix缺失较多，可能是不存在，而Zipcode缺失较少，且邮编这个东西比较重要，应该是存在但是没有被记录

## 4、去除缺失值

In [25]:
#  移除所有包含缺失值的行
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


In [27]:
#  移除包含缺失值的列
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009


In [28]:
#  计算丢失了多少数据
print('数据原有列：%d\n' % nfl_data.shape[1])
print('数据非空列：%d\n' % columns_with_na_dropped.shape[1])

数据原有列：102

数据非空列：41



In [4]:
#  移除包含缺失值的行
sf_permits.dropna(axis=1)


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992
5,201706149344,8,otc alterations permit,06/14/2017,4105,009,800,Indiana,issued,07/06/2017,06/14/2017,1466911170855
6,201706300814,8,otc alterations permit,06/30/2017,1739,020,1291,11th,complete,07/12/2017,06/30/2017,1468970403692
7,M803667,8,otc alterations permit,06/30/2017,4789,014,1465,Revere,issued,06/30/2017,06/30/2017,1469035175050
8,M804227,8,otc alterations permit,07/05/2017,1212,054,2094,Fell,issued,07/05/2017,07/05/2017,1469198253772
9,M804767,8,otc alterations permit,07/06/2017,1259,016,89,Alpine,issued,07/06/2017,07/06/2017,146932394171


In [5]:
#  移除包含缺失值的列
sf_columns_with_na_dropped = sf_permits.dropna(axis=1)
sf_columns_with_na_dropped.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


## 5、填充缺失值

In [10]:
#  拿nfl数据的一部分
sub_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
sub_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


In [8]:
#  将缺失值替换为0
sub_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [11]:
#  将缺失值替换为下一行的值，如果没有下一行则替换为0
sub_nfl_data.fillna(method='bfill',axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [12]:
sub_permits = sf_permits.loc[:,"Street Number Suffix":"Issued Date"].head()
sub_permits

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date
0,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015
1,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017
2,NaN,Pacific,Av,NaN,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,NaN
3,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017
4,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017


In [13]:
sub_permits.fillna(100)

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date
0,100,Ellis,St,100.0,100,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015
1,100,Geary,St,0.0,100,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017
2,100,Pacific,Av,100.0,100,installation of separating wall,withdrawn,09/26/2017,05/27/2016,100
3,100,Pacific,Av,0.0,100,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017
4,100,Market,St,100.0,100,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017


In [14]:
sub_permits.fillna(method='bfill',axis=0).fillna(100)

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date
0,100.0,Ellis,St,0.0,100.0,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015
1,100.0,Geary,St,0.0,100.0,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017
2,100.0,Pacific,Av,0.0,100.0,installation of separating wall,withdrawn,09/26/2017,05/27/2016,07/18/2017
3,100.0,Pacific,Av,0.0,100.0,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017
4,100.0,Market,St,100.0,100.0,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017
